# 🧠 VM Placement Using MORL and Heuristic Algorithms

This Python script simulates the **Virtual Machine (VM) placement problem** using:

- 🧩 **Heuristic Algorithms**: Best Fit, Worst Fit
- 🎯 **Multi-Objective Reinforcement Learning (MORL)**: Optimized via Chebyshev scalarization and Q-learning

## 🎯 Objectives
Minimize:
- ⚡ **Energy Consumption** — Based on PM power profiles
- 🧮 **Resource Wastage** — Difference between allocated and available CPU/memory

## 🧱 Modules Overview
- **`PM`**: Physical Machine with CPU, memory, and power usage
- **`VM`**: Virtual Machine with CPU and memory demands
- **`VMPEnvironment`**: Simulates VM-to-PM placement dynamics
- **`VMPMORLAgent`**: Learns placement using Chebyshev scalarization
- **`train_vmpmorl`**: Trains the MORL agent using Q-learning
- **`best_fit`, `worst_fit`**: Greedy heuristics for placement
- **`simulate_algorithm`**: Executes and evaluates strategies

## 🚀 What This Script Does
1. Trains a MORL agent on 10 VMs and 10 PMs
2. Applies Best Fit and Worst Fit heuristics
3. Compares energy & resource wastage via bar plots

## 📊 Output
Visual comparison of strategies:
- Energy efficiency
- Resource utilization


In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from collections import defaultdict

class PM:
    """Physical Machine (PM) with CPU and memory resources."""
    def __init__(self, cpu_cap, mem_cap, idle_power=120, full_power=185):
        self.cpu_cap = cpu_cap
        self.mem_cap = mem_cap
        self.cpu_used = 0.0
        self.mem_used = 0.0
        self.idle_power = idle_power
        self.full_power = full_power
        self.vm_list = []  # Track VMs assigned to this PM

    def utilization(self):
        """Return normalized CPU and memory utilization."""
        return (self.cpu_used / self.cpu_cap, self.mem_used / self.mem_cap)

    def add_vm(self, vm_id, vm_cpu, vm_mem):
        """Attempt to place a VM on this PM. Returns True if successful."""
        new_cpu = self.cpu_used + vm_cpu
        new_mem = self.mem_used + vm_mem
        if new_cpu <= self.cpu_cap and new_mem <= self.mem_cap:
            self.cpu_used = new_cpu
            self.mem_used = new_mem
            self.vm_list.append((vm_id, vm_cpu, vm_mem))  # Store VM ID and resource usage
            return True
        return False

class VM:
    """Virtual Machine (VM) with CPU and memory demands."""
    def __init__(self, vm_id, cpu_demand, mem_demand):
        self.id = vm_id
        self.cpu = cpu_demand
        self.mem = mem_demand

class VMPEnvironment:
    """Environment for VM placement."""
    def __init__(self, pm_list, vm_list, threshold=0.9):
        self.pms = pm_list
        self.vms = vm_list
        self.threshold = threshold
        self.current_step = 0

    def reset(self):
        """Reset environment for a new episode."""
        self.current_step = 0
        for pm in self.pms:
            pm.cpu_used = 0.0
            pm.mem_used = 0.0
            pm.vm_list = []  # Clear VM list
        return self._get_state()

    def _get_state(self):
        """Return state as normalized CPU and memory utilization for all PMs."""
        state = []
        for pm in self.pms:
            cpu_util, mem_util = pm.utilization()
            state.extend([cpu_util, mem_util])
        return np.array(state)

    def step(self, action):
        """Place VM on selected PM. Return next state, rewards, done."""
        vm = self.vms[self.current_step]
        pm = self.pms[action]

        # Check if PM can accommodate VM
        success = pm.add_vm(vm.id, vm.cpu, vm.mem)
        if not success:
            raise ValueError("Invalid action: PM cannot accommodate VM.")

        # Calculate rewards
        energy_reward = self._energy_reward(pm)
        wastage_reward = self._wastage_reward(pm)

        self.current_step += 1
        done = (self.current_step >= len(self.vms))
        next_state = self._get_state()

        return next_state, [energy_reward, wastage_reward], done, action

    def _energy_reward(self, pm):
        """Reward based on energy consumption."""
        if pm.cpu_used == 0 and pm.mem_used == 0:
            return 0
        utilization = pm.cpu_used / pm.cpu_cap
        energy = (pm.full_power - pm.idle_power) * utilization + pm.idle_power
        return -energy  # Negative because we want to minimize

    def _wastage_reward(self, pm):
        """Reward based on resource wastage."""
        residual_cpu = self.threshold - (pm.cpu_used / pm.cpu_cap)
        residual_mem = self.threshold - (pm.mem_used / pm.mem_cap)
        numerator = abs(residual_cpu - residual_mem)
        denominator = (pm.cpu_used / pm.cpu_cap) + (pm.mem_used / pm.mem_cap) + 0.0001
        wastage = numerator / denominator
        return -wastage  # Negative to minimize

class VMPMORLAgent:
    """Multi-objective RL agent with Chebyshev scalarization."""
    def __init__(self, state_size, action_size, lr=0.1, gamma=0.8, epsilon=0.1):
        self.lr = lr
        self.gamma = gamma
        self.epsilon = epsilon
        self.weights = [0.5, 0.5]  # Equal weight for energy and wastage
        self.z_star = [0, 0]  # Utopian point (initialized to 0)

        # Q-tables for each objective
        self.q_energy = defaultdict(lambda: np.zeros(action_size))
        self.q_wastage = defaultdict(lambda: np.zeros(action_size))

    def chebyshev_scalarization(self, rewards):
        """Compute Chebyshev scalarized value."""
        term1 = self.weights[0] * abs(rewards[0] - self.z_star[0])
        term2 = self.weights[1] * abs(rewards[1] - self.z_star[1])
        return max(term1, term2)

def train_vmpmorl(pm_list, vm_list, episodes=500):
    env = VMPEnvironment(pm_list, vm_list)
    agent = VMPMORLAgent(state_size=2*len(pm_list), action_size=len(pm_list))
    pareto_front = []
    best_pm_vm_mapping = {}

    for episode in range(episodes):
        state = env.reset()
        done = False
        total_rewards = [0, 0]

        while not done:
            available_actions = [i for i, pm in enumerate(env.pms)
                                 if pm.cpu_used + env.vms[env.current_step].cpu <= pm.cpu_cap and
                                 pm.mem_used + env.vms[env.current_step].mem <= pm.mem_cap]
            action = random.choice(available_actions)
            next_state, rewards, done, assigned_pm = env.step(action)

            total_rewards[0] += rewards[0]
            total_rewards[1] += rewards[1]

        # Add to Pareto front if not dominated
        is_dominated = any(sol[0] <= total_rewards[0] and sol[1] <= total_rewards[1] for sol in pareto_front)
        if not is_dominated:
            pareto_front.append(total_rewards)

            # Store best VM to PM mapping
            best_pm_vm_mapping = {i: pm.vm_list for i, pm in enumerate(env.pms) if pm.vm_list}

    # Applying Chebyshev Scalarization to select the best solution
    best_solution = min(pareto_front, key=agent.chebyshev_scalarization)

    return agent, pareto_front, best_solution, best_pm_vm_mapping

# Heuristic Algorithms: Best Fit, Worst Fit, First Fit
def best_fit(pms, vm):
    """Best Fit: Place VM in PM with least remaining space after placement."""
    min_space = float('inf')
    best_pm = -1
    for i, pm in enumerate(pms):
        if pm.cpu_used + vm.cpu <= pm.cpu_cap and pm.mem_used + vm.mem <= pm.mem_cap:
            remaining_space = (pm.cpu_cap - (pm.cpu_used + vm.cpu)) + (pm.mem_cap - (pm.mem_used + vm.mem))
            if remaining_space < min_space:
                min_space = remaining_space
                best_pm = i
    return best_pm

def worst_fit(pms, vm):
    """Worst Fit: Place VM in PM with most remaining space after placement."""
    max_space = -1
    worst_pm = -1
    for i, pm in enumerate(pms):
        if pm.cpu_used + vm.cpu <= pm.cpu_cap and pm.mem_used + vm.mem <= pm.mem_cap:
            remaining_space = (pm.cpu_cap - (pm.cpu_used + vm.cpu)) + (pm.mem_cap - (pm.mem_used + vm.mem))
            if remaining_space > max_space:
                max_space = remaining_space
                worst_pm = i
    return worst_pm

def simulate_algorithm(pm_list, vm_list, algorithm_func):
    """Simulate VM placement with given heuristic algorithm."""
    for pm in pm_list:
        pm.cpu_used = 0.0
        pm.mem_used = 0.0
        pm.vm_list = []

    for vm in vm_list:
        selected_pm_idx = algorithm_func(pm_list, vm)
        if selected_pm_idx == -1:
            raise ValueError(f"VM {vm.id} could not be placed using {algorithm_func.__name__}.")
        pm_list[selected_pm_idx].add_vm(vm.id, vm.cpu, vm.mem)

    total_energy = sum((pm.full_power - pm.idle_power) * (pm.cpu_used / pm.cpu_cap) + pm.idle_power
                       for pm in pm_list if pm.cpu_used > 0)
    total_wastage = sum(abs((pm.cpu_cap - pm.cpu_used) - (pm.mem_cap - pm.mem_used)) /
                        ((pm.cpu_used / pm.cpu_cap) + (pm.mem_used / pm.mem_cap) + 0.0001)
                        for pm in pm_list if pm.cpu_used > 0)

    return total_energy, total_wastage

if __name__ == "__main__":
    # Example setup
    pms = [PM(1.0, 1.0) for _ in range(10)]  # 10 PMs
    vms = [VM(i, random.uniform(0.2, 0.4), random.uniform(0.2, 0.4)) for i in range(10)]  # 10 VMs
    for vm in vms:
        print(f"VM {vm.id} -> CPU: {vm.cpu:.2f}, Memory: {vm.mem:.2f}")

    # Train MORL agent
    agent, pareto_front, best_solution, best_pm_vm_mapping = train_vmpmorl(pms, vms, episodes=100)

    # Simulate heuristic algorithms
    bf_energy, bf_wastage = simulate_algorithm(pms, vms, best_fit)
    wf_energy, wf_wastage = simulate_algorithm(pms, vms, worst_fit)

    # MORL results
    morl_energy = -best_solution[0]
    morl_wastage = -best_solution[1]

    # Plotting results
    algorithms = ['Best Fit', 'Worst Fit', 'MORL']
    energy_values = [bf_energy, wf_energy, morl_energy]
    wastage_values = [bf_wastage, wf_wastage, morl_wastage]

    # Bar Plot for Energy and Wastage
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))

    # Energy plot
    ax[0].bar(algorithms, energy_values, color=['blue', 'orange', 'green'])
    ax[0].set_title('Energy Consumption Comparison')
    ax[0].set_ylabel('Energy (W)')
    ax[0].set_xlabel('Algorithm')

    # Wastage plot
    ax[1].bar(algorithms, wastage_values, color=['blue', 'orange', 'green'])
    ax[1].set_title('Resource Wastage Comparison')
    ax[1].set_ylabel('Wastage (%)')
    ax[1].set_xlabel('Algorithm')

    plt.tight_layout()
    plt.show()

